## Table of Contents

- [Modern Portfolio Theory](#modern-portfolio-theory)
  - [Data Collection](#data-collection)
  - [Visualize Time Series](#visualize-time-series)
  - [Portfolio Composition](#portfolio-composition)
- [Portfolio Statistics](#portfolio-statistics)
  - [Portfolio Simulation](#portfolio-simulation)
  - [Maximum Sharpe Portfolio](#maximum-sharpe-portfolio)
  - [Visualize Simulated Portfolio](#visualize-simulated-portfolio)
- [Efficient Frontier](#efficient-frontier)
  - [Constrained Optimization](#constrained-optimization)
  - [Portfolio Statistics](#portfolio-statistics)
  - [Efficient Frontier Portfolio](#efficient-frontier-portfolio)
- [References](#references)


## Modern Portfolio Theory

This groundbreaking theory in finance also goes by Mean-Variance Portfolio Theory (MVP). Under the premise that returns are normally distributed, we can leverage the examination of the mean and variance of said returns to detail the distribution of end-of-period wealth. 

This theory introduces the idea of constructing a diversified portfolio that minimzes risk or maximizes portfolio returns. Extending this idea is the idea of an **Efficient Frontier** that represents a set of optimal portfolios under the risk-retun spectrum; portfolios along the upper frontier curve are most optimal, while those under the curve are sub-optimal portfolio constructions. 

To further elaborate, the various portfolio constructions sitting on the frontier offer:
- The highest expected return for a given level of risk
- The lowest level of risk for a given level of expected returns

With such a curve, an investor's goal should be to select a level of risk that satisfies their risk appetite, and then proceed to find a portfolio makeup that maximizes returns based on the chosen risk level. 

In [24]:
import pandas as pd 
import yfinance as yf

import numpy as np 
from numpy import *
from numpy.linalg import multi_dot

import cufflinks as cf 
cf.set_config_file(offline=True, dimensions=((1000,600)))

import plotly.express as px 
px.defaults.width, px.defaults.height = 1000, 600

pd.set_option('display.precision', 4)


import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.precision', 4)

### Data Collection
We now collect stock price data fromour stock list that will make up our portfolio



In [25]:
# international etf portfolio tickers
assets = ['META', 'GOOG', 'NFLX', 'AMZN', 'AAPL']
assets.sort()

# number of assets 
num_of_assets = len(assets)

# number of portfolios for optimization

num_of_ports = 5000

In [26]:
# get yfinance tickers

yahoo_tickers = [x for x in assets]

# collect and read data for multiple stocks at once
df = yf.download(yahoo_tickers, start='2023-01-01', end='2024-04-03', progress=False)['Adj Close']
df.columns = assets
df

,AAPL,AMZN,GOOG,META,NFLX
Date,,,,,
2023-01-03,124.0480,85.82,89.70,124.6078,294.95
2023-01-04,125.3275,85.14,88.71,127.2350,309.41
2023-01-05,123.9985,83.12,86.77,126.8055,309.70
2023-01-06,128.5609,86.08,88.16,129.8822,315.55
2023-01-09,129.0865,87.36,88.80,129.3328,315.17
...,...,...,...,...,...
2024-03-26,169.4801,178.30,151.70,495.8900,629.24
2024-03-27,173.0752,179.83,151.94,493.8600,613.53
2024-03-28,171.2477,180.38,152.26,485.5800,607.33


### Visualize Time Series



In [27]:
# plot historical prices
df['2023':].normalize().iplot(kind='line', title="Normalized Price Plot")

In [28]:
# df of returns and volatility
returns = df.pct_change().dropna()
df1 = pd.DataFrame({
    'Annual Return': round(returns.mean() * 252 * 100, 2),
    'Annual Volatility': round(returns.std() * np.sqrt(252) * 100, 2) 
})

df1

,Annual Return,Annual Volatility
AAPL,26.84,20.23
AMZN,65.20,31.74
GOOG,49.18,30.02
META,120.17,41.71
NFLX,65.79,36.49


In [29]:
# plot annualized returns and volatility 

df1.iplot(
    kind='bar',
    title='Annualized Return and Volatility (%)',
    shared_xaxes=True,
    orientation="h"
)

In [30]:
df1.reset_index().iplot(
    kind='pie',
    title='Annualized Return (%)',
    labels='index',
    values='Annual Return',
    textinfo='percent+label',
    hole=0.4
)

## Portfolio Statistics
Consider a portfolio that is fully invested in only risky assets. Let $w$ and $\mu$ be the vector of weights and mean returns of $n$ assets respectively:

$$
\boldsymbol{w} = \begin{pmatrix}
w_1 \\
w_2 \\
\vdots \\
w_n
\end{pmatrix}
; \quad
\boldsymbol{\mu} = \begin{pmatrix}
\mu_1 \\
\mu_2 \\
\vdots \\
\mu_n
\end{pmatrix}
$$

where the $\sum_{i=1}^n \boldsymbol{w_i} = 1$

**Expected Portfolio Return** is then the dot product of the expected returns and their associated weights:

$$\mu_\pi = \boldsymbol{w^T} \cdot \boldsymbol{\mu}$$

which is equivalent to $\sum_{i=1}^n {w_i}\mu_i$

**Expected Portfolio Variance** is then the multidot product of weights and the covariance matrix:

$$\sigma_{\pi}^2 = \boldsymbol{w^T} \cdot \boldsymbol{\Sigma} \cdot \boldsymbol{w}$$

where, the covariance matrix $\Sigma$ is:

$$
\Sigma = \begin{pmatrix}
\Sigma_{1,1} & \cdots & \Sigma_{1,n} \\
\vdots & \ddots & \vdots \\
\Sigma_{n,1} & \cdots & \Sigma_{n,n}
\end{pmatrix}
$$


### Portfolio Simulation
We will now enforce Monte Carlo simulation to randomly generate portfolio weights on a large scaler to then compute the expected portfolio return, variance, Sharpe ratio for each simulated allocation. After this step, we then will locate the porfolio the highest return per unit of risk.

In [31]:
def portfolio_simulation(returns):

    # Initialize the lists
    rets = []
    vols = []
    wts = []

    # Simulate 5,000 portfolios
    for i in range (num_of_ports):
        
        # Generate random weights
        weights = random.random(num_of_assets)[:, newaxis]
        
        # Set weights such that sum of weights equals 1
        weights /= sum(weights)
        
        # Portfolio statistics
        rets.append(weights.T @ array(returns.mean() * 252)[:, newaxis])        
        vols.append(sqrt(multi_dot([weights.T, returns.cov()*252, weights])))
        wts.append(weights.flatten())

    # Create a dataframe for analysis
    portdf = 100*pd.DataFrame({
        'portfolio_return': array(rets).flatten(),
        'portfolio_volatilities': array(vols).flatten(),
        'weights': list(array(wts))
        })
    
    portdf['sharpe_ratio'] = portdf['portfolio_return'] / portdf['portfolio_volatilities']

    return round(portdf,2)

### Maximum Sharpe Portfolio

In [32]:
# create df for analysis

temp = portfolio_simulation(returns)
temp.head()

,portfolio_return,portfolio_volatilities,weights,sharpe_ratio
0,71.77,27.17,"[1.7322448689481251, 38.72158111807029, 29.797...",2.64
1,76.33,28.01,"[2.213685549575592, 41.44773044469017, 21.3146...",2.73
2,62.71,23.62,"[22.173625404531762, 7.320538898006176, 23.884...",2.66
3,65.89,25.00,"[19.08861612635194, 0.02545660560317972, 41.00...",2.64
4,66.86,24.68,"[28.61416813752212, 23.08037753542028, 17.0140...",2.71


In [33]:
# fetch the maximum sharpe ratio portfolio
temp.iloc[temp.sharpe_ratio.idxmax()]

portfolio_return                                                      92.58
portfolio_volatilities                                                 30.7
weights                   [1.2332467172158628, 16.346123275329262, 8.028...
sharpe_ratio                                                           3.02
Name: 3127, dtype: object

In [34]:
# verify the above result
temp.describe().T

,count,mean,std,min,25%,50%,75%,max
portfolio_return,5000.0,65.4117,8.7557,37.74,59.225,65.47,71.1825,103.85
portfolio_volatilities,5000.0,24.6967,1.8556,19.70,23.330,24.54,25.8700,34.69
sharpe_ratio,5000.0,2.6393,0.1934,1.83,2.510,2.68,2.7900,3.02


In [35]:
# max sharpe ratio portfolio weights or "msrpwts" for short

msrpwts = temp['weights'][temp['sharpe_ratio'].idxmax()]

# allocation to attain max sharpe ratio portfolio
dict(zip(assets, around(msrpwts, 2)))

{'AAPL': 1.23, 'AMZN': 16.35, 'GOOG': 8.03, 'META': 52.76, 'NFLX': 21.63}

### Visualize Simulated Portfolio


In [36]:
# Plot simulated portfolio

fig = px.scatter(
    temp, x='portfolio_volatilities', y='portfolio_return', color='sharpe_ratio', 
    labels={'portfolio_volatilities': 'Expected Volatility', 'portfolio_return': 'Expected Return','sharpe_ratio': 'Sharpe Ratio'},
    title="Monte Carlo Simulated Portfolio"
     ).update_traces(mode='markers', marker=dict(symbol='cross'))

# Plot max sharpe 

fig.add_scatter(
    mode='markers', 
    x=[temp.iloc[temp.sharpe_ratio.idxmax()]['portfolio_volatilities']], 
    y=[temp.iloc[temp.sharpe_ratio.idxmax()]['portfolio_return']], 
    marker=dict(color='RoyalBlue', size=20, symbol='star'),
    name = 'Max Sharpe'
).update(layout_showlegend=False)

# Show spikes

fig.update_xaxes(showspikes=True)
fig.update_yaxes(showspikes=True)
fig.show()

## Efficient Frontier

The Efficient Frontier represents a set of portfolio options that balance the trade-off between achieving the highest possible return for a given level of risk and minimizing volatility for a set level of return.


**Return objective:**

    Minimize portfolio variance
$$ \min_{w_1,w_2,\ldots,w_n} \sigma_{p}^2(w_1, w_2, \ldots, w_n) $$
Subject to:
$$ E[R_p] = m $$

**Risk constraint:**

    Maximize expected portfolio return
$$ \max_{w_1,w_2,\ldots,w_n} E[R_p(w_1, w_2, \ldots, w_n)] $$
Subject to:
$$ \sigma_{p}^2(w_1, w_2, \ldots, w_n) = v^2 $$


## Constrained Optimization
The development of optimal portfolios entails an optimization problem with constraints, where boundary conditions and limitations are defined. In this context, the goal is to obtain a function that yields the highest Sharpe ratio and the least variance, with the portfolio weights serving as the variables under consideration. To meet our aim, we will employ the 'minimize' function from the 'scipy.optimize' library.

In [37]:
# Import optimization module from scipy
import scipy.optimize as sco

### Portfolio Statistics
Let's encapsulate the essential statistics within a function to facilitate the optimization process.

In [38]:
def portfolio_stats(weights):
    
    weights = array(weights)[:,newaxis]
    port_rets = weights.T @ array(returns.mean() * 252)[:,newaxis]    
    port_vols = sqrt(multi_dot([weights.T, returns.cov() * 252, weights])) 
    
    return np.array([port_rets, port_vols, port_rets/port_vols]).flatten()


# Minimize the volatility
def min_volatility(weights):
    return portfolio_stats(weights)[1]

# Minimize the variance
def min_variance(weights):
    return portfolio_stats(weights)[1]**2

# Maximizing sharpe ratio
def max_sharpe_ratio(weights):
    return -portfolio_stats(weights)[2]

### Efficient Frontier Portfolio
For efficient frontier portfolios, we fix a target return and derice the objective function.

In [39]:
# Specify constraints, bounds and initial weights

cons = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
bnds = tuple((0, 1) for x in range(num_of_assets))
initial_wts = num_of_assets*[1./num_of_assets]

In [40]:
# Optimizing for maximum sharpe ratio
opt_sharpe = sco.minimize(max_sharpe_ratio, initial_wts, method='SLSQP', bounds=bnds, constraints=cons)

# Optimizing for minimum variance
opt_var = sco.minimize(min_variance, initial_wts, method='SLSQP', bounds=bnds, constraints=cons)

In [41]:
# Efficient Frontier
# targetrets = linspace(0.01,0.11,100)

targetrets = linspace(0.155,0.24,100)
tvols = []

for tr in targetrets:
    
    ef_cons = ({'type': 'eq', 'fun': lambda x: portfolio_stats(x)[0] - tr},
               {'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    
    opt_ef = sco.minimize(min_volatility, initial_wts, method='SLSQP', bounds=bnds, constraints=ef_cons)
    
    tvols.append(opt_ef['fun'])

targetvols = array(tvols)

In [42]:
# Create EF Dataframe for plotting

efport = pd.DataFrame({
    'targetrets' : around(100*targetrets,2),
    'targetvols': around(100*targetvols,2),
    'targetsharpe': around(targetrets/targetvols,2)
})

efport.head()

,targetrets,targetvols,targetsharpe
0,15.50,20.23,0.77
1,15.59,20.23,0.77
2,15.67,20.23,0.77
3,15.76,20.23,0.78
4,15.84,20.23,0.78


In [43]:
# Plot efficient frontier portfolio
fig = px.scatter(
    efport, x='targetvols', y='targetrets',  color='targetsharpe',
    labels={'targetrets': 'Expected Return', 'targetvols': 'Expected Volatility','targetsharpe': 'Sharpe Ratio'},
    title="Efficient Frontier Portfolio"
     ).update_traces(mode='markers', marker=dict(symbol='cross'))


# Plot maximum sharpe portfolio
fig.add_scatter(
    mode='markers',
    x=[100*portfolio_stats(opt_sharpe['x'])[1]], 
    y=[100*portfolio_stats(opt_sharpe['x'])[0]],
    marker=dict(color='red', size=20, symbol='star'),
    name = 'Max Sharpe'
).update(layout_showlegend=False)

# Plot minimum variance portfolio
fig.add_scatter(
    mode='markers',
    x=[100*portfolio_stats(opt_var['x'])[1]], 
    y=[100*portfolio_stats(opt_var['x'])[0]],
    marker=dict(color='green', size=20, symbol='star'),
    name = 'Min Variance'
).update(layout_showlegend=False)

# Show spikes
fig.update_xaxes(showspikes=True)
fig.update_yaxes(showspikes=True)
fig.show()

## References
- [YFinance Documentation](https://urldefense.com/v3/__https://github.com/ranaroussi/yfinance__;!!KGvANbslH1YjwA!6YHgIqFojDyBL2UWhwuqkXOwdoK2vVZy_nqVQRrANDJWW8M9ozuqJyRQEW-JUPaQxkPY_gcida-1eBJ3le2LumK9zHyGeV7vwhEXAuk$)
- [Numpy Linear Algebra](https://urldefense.com/v3/__https://numpy.org/doc/stable/reference/routines.linalg.html__;!!KGvANbslH1YjwA!6YHgIqFojDyBL2UWhwuqkXOwdoK2vVZy_nqVQRrANDJWW8M9ozuqJyRQEW-JUPaQxkPY_gcida-1eBJ3le2LumK9zHyGeV7vswXOYqg$)
- [Python Resources](https://urldefense.com/v3/__https://github.com/kannansingaravelu/PythonResources__;!!KGvANbslH1YjwA!6YHgIqFojDyBL2UWhwuqkXOwdoK2vVZy_nqVQRrANDJWW8M9ozuqJyRQEW-JUPaQxkPY_gcida-1eBJ3le2LumK9zHyGeV7v3wl6uEk$)
- [Styling Plotly Express Figures](https://urldefense.com/v3/__https://plotly.com/python/styling-plotly-express__;!!KGvANbslH1YjwA!6YHgIqFojDyBL2UWhwuqkXOwdoK2vVZy_nqVQRrANDJWW8M9ozuqJyRQEW-JUPaQxkPY_gcida-1eBJ3le2LumK9zHyGeV7vjdS1JuY$)
- [Scipy Optimization](https://urldefense.com/v3/__https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html__;!!KGvANbslH1YjwA!6YHgIqFojDyBL2UWhwuqkXOwdoK2vVZy_nqVQRrANDJWW8M9ozuqJyRQEW-JUPaQxkPY_gcida-1eBJ3le2LumK9zHyGeV7vOaFSrk0$)